In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import recall_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random
import lightgbm as lgb

In [ ]:
!pip install lightning

   ---------------------------------------- 0.0/811.0 kB ? eta -:--:--
   --------------------------------------- 811.0/811.0 kB 11.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/890.6 kB ? eta -:--:--
   --------------------------------------- 890.6/890.6 kB 39.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/815.2 kB ? eta -:--:--
   --------------------------------------- 815.2/815.2 kB 34.6 MB/s eta 0:00:00


In [ ]:
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import torch

from replay.metrics import OfflineMetrics, Recall, Precision, MAP, NDCG, HitRate, MRR
from replay.metrics.torch_metrics_builder import metrics_to_df
from replay.splitters import LastNSplitter
from replay.utils import get_spark_session
from replay.data import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
    Dataset,
)
from replay.models.nn.optimizer_utils import FatOptimizerFactory
from replay.models.nn.sequential.callbacks import (
    ValidationMetricsCallback,
    SparkPredictionCallback,
    PandasPredictionCallback,
    TorchPredictionCallback,
    QueryEmbeddingsPredictionCallback
)
from replay.models.nn.sequential.postprocessors import RemoveSeenItems
from replay.data.nn import (
    SequenceTokenizer,
    SequentialDataset,
    TensorFeatureSource,
    TensorSchema,
    TensorFeatureInfo
)
from replay.models.nn.sequential import SasRec
from replay.models.nn.sequential.sasrec import (
    SasRecPredictionDataset,
    SasRecTrainingDataset,
    SasRecValidationDataset,
    SasRecPredictionBatch,
    SasRecModel
)

import pandas as pd

In [ ]:
# Загрузка данных
user_features = pd.read_csv('data/user_features.csv')
item_features = pd.read_csv('data/item_features.csv')
interactions  = pd.read_csv('data/events.csv')

In [ ]:
interactions

,user_id,item_id,rating,timestamp
0,0,1505,4,0
1,0,3669,3,1
2,0,584,4,2
3,0,3390,3,3
4,0,2885,4,4
...,...,...,...,...
894144,6039,1453,4,223
894145,6039,613,5,224
894146,6039,1548,4,225
894147,6039,241,4,226


In [ ]:
interactions["timestamp"] = interactions["timestamp"].astype("int64")
interactions = interactions.sort_values(by="timestamp")
interactions["timestamp"] = interactions.groupby("user_id").cumcount()

In [ ]:
splitter = LastNSplitter(
    N=1,
    divide_column="user_id",
    query_column="user_id",
    strategy="interactions",
)

raw_test_events, raw_test_gt = splitter.split(interactions)
raw_validation_events, raw_validation_gt = splitter.split(raw_test_events)
raw_train_events = raw_validation_events

In [ ]:
def prepare_feature_schema(is_ground_truth: bool) -> FeatureSchema:
    base_features = FeatureSchema(
        [
            FeatureInfo(
                column="user_id",
                feature_hint=FeatureHint.QUERY_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="item_id",
                feature_hint=FeatureHint.ITEM_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
        ]
    )
    if is_ground_truth:
        return base_features

    all_features = base_features + FeatureSchema(
        [
            FeatureInfo(
                column="timestamp",
                feature_type=FeatureType.NUMERICAL,
                feature_hint=FeatureHint.TIMESTAMP,
            ),
        ]
    )
    return all_features

In [ ]:
train_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_train_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)

In [ ]:
validation_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_validation_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
validation_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_validation_gt,
    check_consistency=True,
    categorical_encoded=False,
)

In [ ]:
test_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_test_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
test_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_test_gt,
    check_consistency=True,
    categorical_encoded=False,
)

In [ ]:
ITEM_FEATURE_NAME = "item_id_seq"

tensor_schema = TensorSchema(
    TensorFeatureInfo(
        name=ITEM_FEATURE_NAME,
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL,
        feature_sources=[TensorFeatureSource(FeatureSource.INTERACTIONS, train_dataset.feature_schema.item_id_column)],
        feature_hint=FeatureHint.ITEM_ID,
    )
)

In [ ]:
tokenizer = SequenceTokenizer(tensor_schema, allow_collect_to_master=True)
tokenizer.fit(train_dataset)

sequential_train_dataset = tokenizer.transform(train_dataset)

sequential_validation_dataset = tokenizer.transform(validation_dataset)
sequential_validation_gt = tokenizer.transform(validation_gt, [tensor_schema.item_id_feature_name])

sequential_validation_dataset, sequential_validation_gt = SequentialDataset.keep_common_query_ids(
    sequential_validation_dataset, sequential_validation_gt
)

In [ ]:
test_query_ids = test_gt.query_ids
test_query_ids_np = tokenizer.query_id_encoder.transform(test_query_ids)["user_id"].values
sequential_test_dataset = tokenizer.transform(test_dataset).filter_by_query_id(test_query_ids_np)

In [ ]:
MAX_SEQ_LEN = 200
BATCH_SIZE = 1024
NUM_WORKERS = 8

model = SasRec(
    tensor_schema,
    block_count=2,
    head_count=2,
    max_seq_len=MAX_SEQ_LEN,
    hidden_size=300,
    dropout_rate=0.5,
    optimizer_factory=FatOptimizerFactory(learning_rate=0.001),
)

csv_logger = CSVLogger(save_dir=".logs/train", name="SASRec_example")

checkpoint_callback = ModelCheckpoint(
    dirpath=".checkpoints",
    save_top_k=1,
    verbose=True,
    # if you use multiple dataloaders, then add the serial number of the dataloader to the suffix of the metric name.
    # For example,"recall@10/dataloader_idx_0"
    monitor="recall@10",
    mode="max",
)

validation_metrics_callback = ValidationMetricsCallback(
    metrics=["map", "ndcg", "recall"],
    ks=[1, 5, 10, 20],
    item_count=train_dataset.item_count,
    postprocessors=[RemoveSeenItems(sequential_validation_dataset)]
)

trainer = L.Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback, validation_metrics_callback],
    logger=csv_logger,
)

train_dataloader = DataLoader(
    dataset=SasRecTrainingDataset(
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)

validation_dataloader = DataLoader(
    dataset=SasRecValidationDataset(
        sequential_validation_dataset,
        sequential_validation_gt,
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=validation_dataloader,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\misha\Documents\PyProj\Recsys_2_new\.venv\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\misha\Documents\PyProj\Recsys_2_new\.checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type             | Params | Mode 
----------------------------------------------------
0 | _model | SasRecModel      | 2.3 M  | train
1 | _loss  | CrossEntropyLoss | 0      | train
----------------------------------------------------
2.3 M     Trainable params
0         Non-trainable params
2.3 M     Total params
9.035     Total estimated model params size (MB)
35        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

C:\Users\misha\Documents\PyProj\Recsys_2_new\.venv\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
C:\Users\misha\Documents\PyProj\Recsys_2_new\.venv\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
C:\Users\misha\Documents\PyProj\Recsys_2_new\.venv\lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Epoch 98, global step 594: 'recall@10' was not in top 1


k              1        10        20         5
map     0.056623  0.115070  0.123696  0.100842
ndcg    0.056623  0.157047  0.188535  0.122316
recall  0.056623  0.296026  0.420695  0.188079



Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 99, global step 600: 'recall@10' was not in top 1
`Trainer.fit` stopped: `max_epochs=100` reached.


k              1        10        20         5
map     0.055132  0.113685  0.122356  0.099945
ndcg    0.055132  0.155527  0.187043  0.121944
recall  0.055132  0.293874  0.418377  0.189404



In [ ]:
best_model = SasRec.load_from_checkpoint(checkpoint_callback.best_model_path)

In [ ]:
prediction_dataloader = DataLoader(
    dataset=SasRecPredictionDataset(
        sequential_test_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

csv_logger = CSVLogger(save_dir=".logs/test", name="SASRec_example")


In [ ]:
TOPK = [10]

postprocessors = [RemoveSeenItems(sequential_test_dataset)]


pandas_prediction_callback = PandasPredictionCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

query_embeddings_callback = QueryEmbeddingsPredictionCallback()

trainer = L.Trainer(
    callbacks=[
        pandas_prediction_callback,
        query_embeddings_callback
    ],
    logger=csv_logger,
    inference_mode=True
)
trainer.predict(best_model, dataloaders=prediction_dataloader, return_predictions=False)


pandas_res = pandas_prediction_callback.get_result()
user_embeddings = query_embeddings_callback.get_result()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\misha\Documents\PyProj\Recsys_2_new\.venv\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
pandas_res

,user_id,item_id,score
0,790,1461,5.982999
0,790,2275,5.597245
0,790,2003,5.02258
0,790,213,5.010365
0,790,1044,4.956807
...,...,...,...
6039,4520,1859,5.945872
6039,4520,1828,5.875895
6039,4520,2609,5.691204
6039,4520,1351,5.575523


In [ ]:
init_args = {"query_column": "user_id", "rating_column": "score"}

In [ ]:
recommendations = tokenizer.query_and_item_id_encoder.inverse_transform(pandas_res)

In [ ]:
recommendations

,user_id,item_id,score
0,0,1461,5.982999
0,0,2275,5.597245
0,0,2003,5.02258
0,0,213,5.010365
0,0,1044,4.956807
...,...,...,...
6039,6039,1859,5.945872
6039,6039,1828,5.875895
6039,6039,2609,5.691204
6039,6039,1351,5.575523


In [ ]:
result_metrics = OfflineMetrics(
    [Recall(TOPK), Precision(TOPK), MAP(TOPK), NDCG(TOPK), MRR(TOPK), HitRate(TOPK)], **init_args)(recommendations, raw_test_gt)

In [ ]:
result_metrics

{'Recall@10': 0.2915562913907285,
 'Precision@10': 0.029155629139072846,
 'MAP@10': 0.11954233680227057,
 'NDCG@10': 0.1596007525131245,
 'MRR@10': 0.11954233680227057,
 'HitRate@10': 0.2915562913907285}

In [ ]:
submission_df = recommendations.groupby('user_id')['item_id'].apply(lambda x: ' '.join(x.astype(str))).reset_index()

In [ ]:
submission_df

,user_id,item_id
0,0,1461 2275 2003 213 1044 3271 1422 1332 2551 3002
1,1,232 1246 1686 1822 3101 1813 3656 2467 2518 350
2,2,2354 1687 382 2774 1947 1781 724 221 234 3687
3,3,487 3562 3390 106 605 989 1560 2515 94 1792
4,4,802 2185 2205 2814 810 174 1801 3035 3030 702
...,...,...
6035,6035,1316 1811 3013 2528 883 1315 2054 3409 707 3153
6036,6036,3692 1859 3142 401 772 494 3418 2470 829 2664
6037,6037,2664 2256 3059 1968 1375 1102 1747 2502 3656 2732
6038,6038,2420 1355 450 700 2305 1893 405 2231 2608 797


In [ ]:
submission_df.to_csv('test_sub_3.csv', index=False)